<a href="https://colab.research.google.com/github/viecooo/movie-recommendation-based-on-reviews/blob/main/Scraping_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.3 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=eadf1423c7eae94b6bd470dd4e848f191cc2cd5824c238ecd7bf52c192c86603
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=626b5a359b2933cd8d9d0d05a7031c2d258c53c028ca34fafdd9ace9897ac76f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from transformers import pipeline
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Kelas movie_scraper untuk mengambil informasi dan review film
class movie_scraper:
    def __init__(self, name):
        self.name = name

    def get_movie_info(self):
        data_url = f"http://www.omdbapi.com/?t={self.name}&apikey=79ce9d13"
        resp = requests.get(data_url)
        data = resp.json()  # Menggunakan json() untuk langsung mengonversi respons ke dictionary
        return data

    def get_movie_reviews(self, id):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        search_url = f"https://www.imdb.com/title/{id}/reviews/?ref_=tt_urv_sm"
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        review_div = soup.find_all('div', class_="ipc-html-content-inner-div")
        review_list = " ".join([review.get_text().strip() for review in review_div])  # Menggunakan list comprehension
        return review_list

    def contextualize_summary(self, summary):
        parser = PlaintextParser.from_string(summary, Tokenizer("english"))
        summarizer = LsaSummarizer()
        raw_summary = ' '.join([str(sentence) for sentence in summarizer(parser.document, 20)])
        pipe = pipeline("summarization", model="abhiramd22/t5-base-finetuned-to-summarize-movie-reviews")
        contextual_summary = pipe(raw_summary, min_length=200, max_length=300)
        return contextual_summary

# Memuat data dari file Excel yang sudah di-scrape sebelumnya
df = pd.read_excel('/content/data1.xlsx')

# Membuat list untuk menyimpan data hasil scraping
movie_data = []

# Iterasi untuk mengambil nama film dari DataFrame
for index, row in df.iterrows():
    movie_name = row['Movie_name']
    print(f"Processing movie: {movie_name}")

    # Membuat objek movie_scraper untuk setiap film
    obj = movie_scraper(movie_name)

    # Mengambil informasi film
    info = obj.get_movie_info()
    id = info.get('imdbID', None)

    if id:
        print(f"IMDb ID: {id}")

        # Mengambil review film
        reviews = obj.get_movie_reviews(id)
        print(f"Number of reviews: {len(reviews)}")

        # Menyusun ringkasan review film
        contextual_summary = obj.contextualize_summary(reviews)
        print(f"Contextual Summary: {contextual_summary}")

        # Menyimpan hasil ke dalam list
        movie_data.append({
            "Movie Name": movie_name,
            "IMDb ID": id,
            "Reviews": reviews,
            "Contextual Summary": contextual_summary
        })
    else:
        print(f"IMDb ID tidak ditemukan untuk film {movie_name}")
        # Menyimpan data dengan informasi kosong
        movie_data.append({
            "Movie Name": movie_name,
            "IMDb ID": None,
            "Reviews": None,
            "Contextual Summary": None
        })

# Membuat DataFrame dari hasil yang sudah dikumpulkan
result_df = pd.DataFrame(movie_data)

# Menyimpan DataFrame ke dalam file CSV
result_df.to_csv('movie_reviews_and_summaries.csv', index=False)
print("Data telah disimpan dalam 'movie_reviews_and_summaries.csv'")

# Membaca file hasil scraping
result_df = pd.read_csv('movie_reviews_and_summaries.csv')

# Menggabungkan data Book1.xlsx dengan hasil scraping
merged_df = pd.merge(df, result_df, left_on='Movie_name', right_on='Movie Name', how='outer')

# Menyimpan hasil penggabungan ke file CSV baru
merged_df.to_csv('merged_movie_data.csv', index=False)
print("Data gabungan telah disimpan dalam 'merged_movie_data.csv'")


Processing movie: Red One
IMDb ID: tt14948432
Number of reviews: 15766


Device set to use cpu


Contextual Summary: [{'summary_text': '"Home Alone" offers a mix of action, comedy, and Sci-Fi elements, but its CGI and character designs aren\'t flawless. Despite its flaws, it\'s a fun Christmas movie with interesting holiday lore and cool gimmicks. Dwayne Johnson\'s performance as Callum is decent, but the film\'s scares come off as somewhat funny and cheesy. While it may alienate under 12s with its language, "Red One" remains a solid and creative take on Christmas movies, despite its lackluster acting and lack of unbiased opinions on its own merit. Overall, the movie\'s unique approach to storytelling and storytelling makes it a must-watch for those seeking a feel-good Christmas movie that\'s not too serious about its impact on the genre\'s impact on its overall impact on audiences. Ultimately, "Rose one" is a standout in the genre.'}]
Processing movie: Venom: The Last Dance
IMDb ID: tt16366836
Number of reviews: 39830


Device set to use cpu


Contextual Summary: [{'summary_text': '"Venom: The Last Dance" offers a mixed bag of entertainment, with some finding it entertaining while others criticize its chaotic plot and lack of character development. Tom Hardy\'s performance shines, delivering a blend of humor, action, and Venom\'s signature appetite for chaos. Despite some emotional moments, the film\'s cardboard cutouts and clumsy pacing detract from its potential as a high-octane ride with a surprising amount of heart. Overall, it\'s a messy buddy road trip in which you must turn your brains off...again, despite its lack of space for development and disjointed storylines, leaving audiences disappointed, leaving viewers disappointed by its lackluster action scenes and rushed bangs, leaving a lasting impression on viewers and a forgettable experience for fans of superheroes. Ultimately, it remains a disappointing entry in the Marvel universe.'}]
Processing movie: Mufasa: The Lion King
IMDb ID: tt13186482
Number of reviews: 27

...

Contextual Summary: [{'summary_text': '"28 days later" offers a mixed bag of zombies and modern themes, though it lacks precedents for other zombie movies. Despite its metaphorical narrative and lack of action sequences, the film delivers a compelling survival plot amidst modern issues. While some find it cartoony and lacking in depth, others appreciate its focus on family dynamics and combating crime. Overall, it\'s a fun gimmicky movie that doesn\'t overshadow its predecessors, offering a believable and frightening story that keeps viewers interested all the way to the end of its runtime despite its lack of pacing issues and a lack of focus on zombies. Ultimately, it may not be a smashing success, but it could lead to a sequel with similar themes and plot points that will keep viewers interested for years to come, making it a hit in this subgenre of horror and scares and intrigue.'}]
Processing movie: Bullet Train
IMDb ID: tt12593682
Number of reviews: 20458


Device set to use cpu


Contextual Summary: [{'summary_text': '"The Wild Ride" is a fun action comedy that features Brad Pitt in a standout role. While the film features Brian Tyree Henry and Aaron Taylor-Johnson, it falls short of its potential. Despite some Japanese backdrops and Celebrity Cameos, the film\'s visuals are stunning. Brad Pitt shines in his lead role, while Joey King and Hiroyuki Sanada shine in their roles. While some may find it entertaining, others will appreciate its fun callbacks and craziness, making it a must-watch for those interested in action comedy and a bit of fun while it\'s also a memorable one-liner and villainous stereotypes. Regardless of its merits, "Wild Ride" remains a compelling and thought-provoking exploration of human connection and the human connection in the human dimension, albeit one-liners and villains alike, despite its flaws.'}]
Processing movie: Grown Ups 2
IMDb ID: tt2191701
Number of reviews: 18108


Device set to use cpu


Contextual Summary: [{'summary_text': '"Grown Ups 2" receives mixed reviews, with some praising its comedic moments and likable characters but criticized for its childish humor and lack of storyline. While Adam Sandler shines in his role, the movie\'s lack of real flow and character development detract from its overall impact. Despite some funny moments, it\'s not for everyone, especially those who enjoy comedies with a strong cast. Ultimately, if you enjoyed the first installment, you\'ll find it a forgettable entry in the franchise, suggesting its lack of originality and a lack of genuine fun for those who appreciate Comedies with no real storyline and laughable characters. Overall, "GROWN UPS 2" fails to deliver on its potential, leaving audiences disappointed and disappointed in the sequel\'s success. Regardless, despite its failure to deliver a satisfying cinematic experience, leaving viewers disappointed by its lackluster performances and lackluster storytelling.'}]
Data telah di